In [ ]:
from __future__ import print_function
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pickle
from astropy.io import fits

# path to the spectrum data files (change if you are getting errors loading data)
path=''

### astr3400 day 2 notebook: analyzing a .FITS spectrum from SDSS-V

#### 1. load spectrum file 

this is a single spectrum file from one fiber of one observation from SDSS-V
the data are in FITS format

In [ ]:
file_name = path+'spec-9391-58072-0898.fits'
f = fits.open(file_name)

In [ ]:
# brief look at what's in the file
# it is mostly in the form of "binary tables" a flexible way of storing different types of data
print(f.info()) 

In [ ]:
# this is an overall description of the file and observation
print(f[0].header)

In [ ]:
# spectral data
# flux: F_\lambda in units of 10^-17 erg cm^-2 s^-1 Angstrom^-1
# loglam: observed wavelength in units of log10 Angstrom
# ivar: pipeline uncertainties on the flux
print(f[1].columns)

In [ ]:
# summary data for the observation and the object used in a summary 
# SDSS-V data product called spAll (Spec All)
# e.g. the location of the object is described by PLUG_RA and PLUG_DEC (RA and Dec in degrees)
print(f[2].columns)

In [ ]:
# let's measure some properties of the continuum spectrum
# the continuum is the flattish part not including discrete atomic transitions
wavelength = 10**f[1].data['loglam']
flux = f[1].data['flux']

In [ ]:
# plot the spectrum
plt.plot(wavelength,flux)
plt.ylim(0,80); plt.xlim(3500,10000)
plt.xlabel(r'wavelength (Angstrom)',fontsize=12)
plt.ylabel(r'flux ($10^{-17}$ erg cm$^{-2}$ s$^{-1}$ $\AA^{-1}$)',fontsize=12)

#### 2. Calculating an approximate SDSS i-band continuum flux in AB magnitudes

In [ ]:
# let's measure (approximately) the SDSS i-band magnitude
# first, find the value of the flux close to 7480 Angstrom, the central observed wavelength of the filter
# we'd like to automate this (for Lab 2), so let's find flux values from 7460-7500 Angstrom and average them
# one way to do this is using the numpy command argmin, which finds the index to minimize an expression
i1 = np.argmin() # complete the code here to find the index closest to the wavelength 7460
i2 = np.argmin() # complete the code here to find the index closest to the wavelength 7500

# now calculate the average value of the flux over the index range i1:i2
f_lambda_avg = XX

# calculate a flux value by multiplying your value by 7480 Angstrom 
flux_i = f_lambda_avg*1e-17

# this is the flux density $F_\nu$ in units of Jy
f_nu_Jy = 3.34e4*7480**2.*flux_i

# using notes from class, write an expression to convert 
# the flux value to i-band magnitude and call it m_i
m_i = XX

# compare to value from file spAll (PSFMAG gives the bands ugriz in order)
print('our value: ',m_i)
print('pipeline value: ',f[2].data['PSFMAG'][0][3])

#### 3. Signal to noise

In [ ]:
f[2].data['PLATESN2']

In [ ]:
# pipeline errors
# calculate the error of your continuum values between i1 and i2 as its standard deviation
# you can use the numpy function np.std for standard deviation
flux_error = np.std()

# median flux value over this range between i1 and i2
flux_median = np.median()

# the signal-to-noise is defined as the median value divided by the error
flux_sn = flux_median/flux_error

# compare to the measured value from the pipeline
print('my SN, pipeline SN: ',flux_sn,f[2].data['PLATESN2'])

#### 4. Curve fitting

In [ ]:
# here is an example of using the scipy function curve_fit for a "Gaussian" function plus a constant level
# you will adapt this code below to extract physical information from the quasar's emission line
from scipy.optimize import curve_fit

# define the function we want to use to describe the data
def gaussian(x, *p):
    A, mu, sigma, B = p
    return A*np.exp(-(x-mu)**2/(2.*sigma**2))+B

# make up some fake data
data_x = np.arange(50)/49.*10.-5.
# here are the true parameters we are going to try to recover with curve_fit
A0 = 10. # maximum value of our Gaussian above a flat line
mu0 = 0. # center of our Gaussian function
sigma0 = 1.2 # standard deviation
B0 = 2. # constant level

p_true = [A0,mu0,sigma0,B0]
# generate fake data using the real parameters but adding random noise 
# (you will not need this part below but will instead use the real data)
data_y = gaussian(data_x,*p_true)+np.random.randn(len(data_x))*1.5

# plot our mock data
plt.plot(data_x,data_y)

# now enter your starting guess for the real parameters based on the plot below 
# (the fitting exercise will be more useful if you do not simply copy the "true" parameters above)
A_guess = XX # maximum value of our Gaussian above a flat line
mu_guess = XX # center of our Gaussian function
sigma_guess = XX # standard deviation
B_guess = XX # constant level
p_guess = [A_guess,mu_guess,sigma_guess,B_guess]

# perform the fit of the model to the data starting from your guess
fit, errors = curve_fit(gaussian, data_x, data_y, p0=p_guess)

# get the model data that curve_fit found
data_fit = gaussian(data_x, *fit)
# plot the model on top of the fake data to see how well it did
plt.plot(data_x, data_fit)

# print the parameters the fit found -- how similar are these to the true values?
# should they be exactly the same? 
# do they remain identical when you generate new data and fit it again?
print('best fitting parameters found by curve_fit: ',fit)

In [ ]:
# Now adapt the code from the previous step now to fit an emission line from the real data

In [ ]:
# define data range for the line
data_wave = wavelength[1100:1550]
data_flux = flux[1100:1550]

# plot the data over this range
plt.plot(data_wave,data_flux)
plt.ylim(0,50); plt.xlim(4700,5120)
plt.xlabel(r'wavelength (Angstrom)',fontsize=12)
plt.ylabel(r'flux ($10^{-17}$ erg cm$^{-2}$ s$^{-1}$ $\AA^{-1}$)',fontsize=12)


# define here your initial guess for the parameters
XX

# fit the data using curve_fit
XX

# Get the fitted curve
XX

# plot your fit -- do you think it worked as expected?
XX

# print the best fitting parameters
XX

# how do your fit parameters compare to what you estimated by eye in the day01 notebook?

#### 5. automated fitting of the C IV emission line when it's in an SDSS-V  spectrum

For lab 2 you're going to write a Python script to fit the C IV emission line in one of these spectrum files and return the best fitting parameters. You can use the code from this notebook, but will also need to figure out whether the line is in the spectrum.

The redshift factor changing the wavelength is given by 

$\lambda_{obs} = (1+z) \lambda_{em}$ 

where $\lambda_{em}$ is the emitted wavelength and $\lambda_{obs}$ is the observed one

How can you use this information along with the pipeline redshift of the quasar to figure out if the C IV line is in the SDSS-V spectrum?

In [ ]:
# spectrum pipeline redshift value
z = f[2].data['Z']
print('redshift measured by pipeline: ',z[0])